In [2]:
import os
os.chdir("C:/Users/UV665AR/OneDrive - EY/Documents/GitHub/khb_engine/app")
import pandas as pd
from pathlib import Path
from reporting import report_basic
from input_handler.load_parameters import load_configuration_file
from input_handler.env_setting import run_setting

configPath = Path(r'C:/Users/UV665AR/OneDrive - EY/Documents/GitHub/khb_engine/run_config_file.json')
c = load_configuration_file(configPath=configPath)
rc = run_setting(run_config=c)
rp_b = report_basic.report_load_info(context=rc)
rp_f = report_basic.report_cond_basic(context=rc)
param = rp_b.load_all_param()

from input_handler import data_preprocessor
dp = data_preprocessor.data_preprocessor(context=rc)

In [3]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [22]:
from input_handler.load_parameters import load_configuration_file
from input_handler.env_setting import run_setting
from pathlib import Path
import pandas as pd
from datetime import datetime
from reporting import report_basic
import numpy as np


def load_bond_data(context, param):
    """
    读取bond breakdown和exchange rate数据
    
    Args:
        context: environment setting
        param: all parameter list
    
    Returns:
        bond_df: bond tab数据
        daily_coupon_df: daily_coupon tab数据
        exchange_rate_df: 每日汇率数据
    """
    # 读取bond breakdown数据
    bond_file_path = context.inDataPath / 'bondbreakdown.xlsx'
    bond_df = pd.read_excel(bond_file_path, sheet_name='bond')
    daily_coupon_df = pd.read_excel(bond_file_path, sheet_name='daily_coupon')
    
    # 读取exchange rate数据
    exchange_rate_file_path = context.inDataPath / 'exchange_rate_table_daily.csv'
    exchange_rate_df = pd.read_csv(exchange_rate_file_path)
    
    # 处理日期格式转换
    # 将Date列从M/D/YYYY格式转换为datetime
    exchange_rate_df['DATE'] = pd.to_datetime(exchange_rate_df['Date'], format='%d/%m/%Y')
    
    return bond_df, daily_coupon_df, exchange_rate_df

bond_df, daily_coupon_df, exchange_rate_df = load_bond_data(rc, param)
print(exchange_rate_df)


           Date Exchange Rate  Unnamed: 2       DATE
0     30/9/2024      3,420.25         NaN 2024-09-30
1     1/10/2024      3,420.46         NaN 2024-10-01
2     2/10/2024      3,420.46         NaN 2024-10-02
3     3/10/2024      3,420.92         NaN 2024-10-03
4     4/10/2024      3,421.43         NaN 2024-10-04
5     5/10/2024      3,421.43         NaN 2024-10-05
6     6/10/2024      3,421.43         NaN 2024-10-06
7     7/10/2024      3,422.08         NaN 2024-10-07
8     8/10/2024      3,422.70         NaN 2024-10-08
9     9/10/2024      3,423.28         NaN 2024-10-09
10   10/10/2024      3,423.93         NaN 2024-10-10
11   11/10/2024      3,424.61         NaN 2024-10-11
12   12/10/2024      3,424.61         NaN 2024-10-12
13   13/10/2024      3,424.61         NaN 2024-10-13
14   14/10/2024      3,426.10         NaN 2024-10-14
15   15/10/2024      3,427.07         NaN 2024-10-15
16   16/10/2024      3,427.88         NaN 2024-10-16
17   17/10/2024      3,428.91         NaN 2024

In [ ]:
run_yymm =  {rc.run_yymm}
print(run_yymm)

def get_exchange_rate(exchange_rate_df, date_value, date_format='%Y%m%d'):
    """
    获取指定日期的汇率
    
    Args:
        exchange_rate_df: 汇率数据DataFrame
        date_value: 日期值（字符串）
        date_format: 日期格式，默认为'%Y%m%d'
    
    Returns:
        float: 汇率值，如果找不到则返回1.0
    """
    try:
        # 将日期字符串转换为datetime对象
        if isinstance(date_value, str):
            date_dt = pd.to_datetime(date_value, format=date_format)
        else:
            date_dt = date_value
        
        # 查找匹配的汇率
        matching_rates = exchange_rate_df[exchange_rate_df['DATE'] == date_dt]['Exchange Rate']
        
        if len(matching_rates) > 0:
            return matching_rates.iloc[0]
        else:
            print(f"Warning: No exchange rate found for date {date_value}")
            return 1.0
            
    except Exception as e:
        print(f"Error getting exchange rate for date {date_value}: {e}")
        return 1.0



run_exchange_rate = get_exchange_rate(exchange_rate_df, run_yymm, '%Y%m%d')

{20241231}
Error getting exchange rate for date {20241231}: 'DATE'


In [45]:
def load_bond_data(context, param):
    # Load bond breakdown data
    bond_file_path = context.inDataPath / 'bondbreakdown.xlsx'
    bond_df = pd.read_excel(bond_file_path, sheet_name='bond')
    daily_coupon_df = pd.read_excel(bond_file_path, sheet_name='daily_coupon')
    
    # Load exchange rate data
    exchange_rate_file_path = context.inDataPath / 'exchange_rate_table_daily.csv'
    exchange_rate_df = pd.read_csv(exchange_rate_file_path)
    
    # Process date format conversion
    # Convert Date column from M/D/YYYY format to datetime
    exchange_rate_df['DATE'] = pd.to_datetime(exchange_rate_df['Date'], format='%d/%m/%Y')
    
    # Ensure Exchange Rate column is numeric type
    exchange_rate_df['Exchange Rate'] = pd.to_numeric(exchange_rate_df['Exchange Rate'], errors='coerce')
 
    return bond_df, daily_coupon_df, exchange_rate_df

bond_df, daily_coupon_df, exchange_rate_df = load_bond_data(rc, param)
print(bond_df)
print(daily_coupon_df)
print(exchange_rate_df)


           CONTRACT_ID            CUST_ID payment_date  interest  \
0          20005001762  00009511800001197          NaT       NaN   
1          20005001767  00009511800001197          NaT       NaN   
2          20005001768  00009511800001197          NaT       NaN   
3          20005001769  00009511800001197          NaT       NaN   
4          20005001782  00009511800001197          NaT       NaN   
5        MM20002001574  00000095038004472          NaT       NaN   
6        MM20002002180  00000095038004472          NaT       NaN   
7        MM20002002058  00009500402173548          NaT       NaN   
8        MM20002002097  00009500402173548          NaT       NaN   
9        MM20002002634  00009511800001197          NaT       NaN   
10                 MR1  00009500401559726          NaT       NaN   
11                 MR2  00009500401559726          NaT       NaN   
12   50000011200000008  00009511800001197          NaT       NaN   
13   50001011200000000  00009511800001197       

In [50]:
def get_exchange_rate(exchange_rate_df, input_date):
    match = exchange_rate_df.loc[exchange_rate_df['DATE'] == input_date, 'Exchange Rate']
    if not match.empty:
        return match.iloc[0]
    else:
        return None

 

run_yymm = rc.run_yymm
prev_yymm = rc.prev_yymm
run_date = datetime.strptime(str(run_yymm), "%Y%m%d").strftime("%Y-%m-%d")
print(run_date)
test2 = get_exchange_rate(exchange_rate_df, run_date)
print(test2)

2024-12-31
3420.25


In [58]:
def merge_ecl_results_with_bond(context, param, ecl_result, ecl_result_prev, bond_df):
    """
    Merge ECL results and bond data
    
    Strategy: Use bond tab as the main table, merge ECL results into it
    This ensures all required bond columns are preserved

    Args:
        context: environment setting
        param: all parameter list
        ecl_result: current ECL results
        ecl_result_prev: previous ECL results
        bond_df: bond tab data

    Returns:
        merged_df: merged data with bond tab as base
    """
    # First add _prev suffix to all columns in ecl_result_prev (except CONTRACT_ID and CUST_ID)
    ecl_result_prev_renamed = ecl_result_prev.copy()
    rename_cols = {}
    for col in ecl_result_prev_renamed.columns:
        if col not in ['CONTRACT_ID', 'CUST_ID']:
            rename_cols[col] = f"{col}_prev"
    
    ecl_result_prev_renamed = ecl_result_prev_renamed.rename(columns=rename_cols)
    
    # Select required columns and merge ECL results
    cols_curr = ['CONTRACT_ID', 'CUST_ID', 'IFRS9_MEAS_TYPE', 'INIT_DATE', 'MAT_DATE', 'DRAWN_BAL_OCY', 'IFRS_AMOUNT_LCY', 'EFF_INT_RT', 'AMORTIZATION_OCY', 'IFRS9_PD_12M_MADJ', 'IFRS9_LGD_12M', 'ECL_FINAL_LCY']
    cols_prev = ['CONTRACT_ID', 'CUST_ID', 'IFRS9_MEAS_TYPE_prev', 'INIT_DATE_prev', 'MAT_DATE_prev', 'DRAWN_BAL_OCY_prev', 'IFRS_AMOUNT_LCY_prev', 'EFF_INT_RT_prev', 'AMORTIZATION_OCY_prev', 'IFRS9_PD_12M_MADJ_prev', 'IFRS9_LGD_12M_prev', 'ECL_FINAL_LCY_prev']
    
    # Ensure columns exist, skip if not
    available_cols_curr = [col for col in cols_curr if col in ecl_result.columns]
    available_cols_prev = [col for col in cols_prev if col in ecl_result_prev_renamed.columns]
    
    df_curr = ecl_result[available_cols_curr].copy()
    df_prev = ecl_result_prev_renamed[available_cols_prev].copy()
    
    # Merge ECL results
    merged_ecl = df_curr.merge(df_prev, on=['CONTRACT_ID', 'CUST_ID'], how='outer')
    
    # Start with bond_df as the base table (preserves all bond columns)
    merged_df = bond_df.copy()
    
    # Merge current ECL results into bond_df
    merged_df = merged_df.merge(df_curr, on=['CONTRACT_ID', 'CUST_ID'], how='left')
    
    # Merge previous ECL results into the result
    merged_df = merged_df.merge(df_prev, on=['CONTRACT_ID', 'CUST_ID'], how='left')
    
    # Ensure numeric columns have correct types
    numeric_columns = [
        'DRAWN_BAL_OCY', 'DRAWN_BAL_OCY_prev', 'IFRS_AMOUNT_LCY', 'IFRS_AMOUNT_LCY_prev',
        'EFF_INT_RT', 'EFF_INT_RT_prev', 'AMORTIZATION_OCY', 'AMORTIZATION_OCY_prev', 
        'interest', 'ECL_FINAL_LCY', 'ECL_FINAL_LCY_prev'
    ]
    
    for col in numeric_columns:
        if col in merged_df.columns:
            merged_df[col] = pd.to_numeric(merged_df[col], errors='coerce')
    
    # # Filter out invalid date data
    # # 1899-12-30 usually indicates invalid dates in Excel
    # if 'INIT_DATE' in merged_df.columns:
    #     merged_df = merged_df[merged_df['INIT_DATE'] != '1899-12-30']
    #     print(f"Filtered out bonds with invalid INIT_DATE, remaining: {len(merged_df)}")

    # if 'MAT_DATE' in merged_df.columns:
    #     merged_df = merged_df[merged_df['MAT_DATE'] != '1899-12-30']
    #     print(f"Filtered out bonds with invalid MAT_DATE, remaining: {len(merged_df)}")

    return merged_df




tab_name = 'Bond_breakdown'
rp_f = report_basic.report_cond_basic(context=rc)
df_conditions = param['Conditions'].query("REPORT_NAME == @tab_name")

path_prev = r"C:\Users\UV665AR\OneDrive - EY\2.KHB\99_Data_Server_out\production\20241231\data\02_result\ECL_calculation_result_files_deal_all.csv"
path_curr = r"C:\Users\UV665AR\OneDrive - EY\2.KHB\99_Data_Server_out\production\20241130\data\02_result\ECL_calculation_result_files_deal_all.csv"

# Read the CSV files
ecl_result_prev = pd.read_csv(path_prev, encoding='utf-8')
ecl_result_curr = pd.read_csv(path_curr, encoding='utf-8')

ecl_result_filter_prev = rp_f.overall_filter(df_conditions, ecl_result_prev)
ecl_result_filter = rp_f.overall_filter(df_conditions, ecl_result_curr)
merged_df = merge_ecl_results_with_bond(rc, param, ecl_result_filter, ecl_result_filter_prev, bond_df)
print(merged_df)

output_path = r"C:\Users\UV665AR\OneDrive - EY\2.KHB\99_Data_Server_out\production\20241231\data\03_report\interim"
file_name = os.path.join(output_path, "merged_ecl_bond.csv")
merged_df.to_csv(file_name, index=False, encoding='utf-8')

           CONTRACT_ID            CUST_ID payment_date  interest  \
0          20005001762  00009511800001197          NaT       NaN   
1          20005001767  00009511800001197          NaT       NaN   
2          20005001768  00009511800001197          NaT       NaN   
3          20005001769  00009511800001197          NaT       NaN   
4          20005001782  00009511800001197          NaT       NaN   
5        MM20002001574  00000095038004472          NaT       NaN   
6        MM20002002180  00000095038004472          NaT       NaN   
7        MM20002002058  00009500402173548          NaT       NaN   
8        MM20002002097  00009500402173548          NaT       NaN   
9        MM20002002634  00009511800001197          NaT       NaN   
10                 MR1  00009500401559726          NaT       NaN   
11                 MR2  00009500401559726          NaT       NaN   
12   50000011200000008  00009511800001197          NaT       NaN   
13   50001011200000000  00009511800001197       

In [60]:
from datetime import datetime
import pandas as pd
import os

def calculate_daily_discount_premium(merged_df, exchange_rate_df, run_yymm, prev_yymm):
    """
    Calculate daily discount/premium for each bond using time series NPV calculation
    """
    # ✅ 保留 datetime 对象
    prev_date = datetime.strptime(str(prev_yymm), "%Y%m%d")
    run_date = datetime.strptime(str(run_yymm), "%Y%m%d")
    
    # 计算总天数
    total_days = (run_date - prev_date).days + 1
    
    # 生成日期范围
    all_dates = [(prev_date + pd.Timedelta(days=day)).strftime('%Y-%m-%d') for day in range(total_days)]
    
    # 初始化每日列
    for date_str in all_dates:
        merged_df[date_str] = 0.0
    
    for idx, row in merged_df.iterrows():
        contract_id = row['CONTRACT_ID']
        cust_id = row['CUST_ID']
        
        drawn_balance = row['DRAWN_BAL_OCY']
        eff_int_rate = row['EFF_INT_RT']
        amortization = row['AMORTIZATION_OCY']
        interest_rate = row['interest']
        
        if pd.isna(drawn_balance) or pd.isna(interest_rate) or pd.isna(eff_int_rate) or pd.isna(amortization):
            continue
        
        interest_receivable = drawn_balance * interest_rate / 360
        npv_current = amortization
        
        try:
            payment_date = pd.to_datetime(row['payment_date']) if pd.notna(row['payment_date']) else prev_date
            row_start_date = max(prev_date, payment_date)
            
            maturity_date = pd.to_datetime(row['MAT_DATE']) if pd.notna(row['MAT_DATE']) else run_date
            row_end_date = min(run_date, maturity_date)
            
            row_num_days = (row_end_date - row_start_date).days + 1
            if row_num_days <= 0:
                row_num_days = 1
                
        except Exception as e:
            print(f"Warning: Error calculating date range for contract {contract_id}: {e}")
            row_num_days = 1
            row_start_date = prev_date
            row_end_date = prev_date
        
        for day in range(row_num_days):
            interest_income = npv_current * eff_int_rate / 360
            daily_discount_premium = interest_receivable - interest_income
            
            npv_next = npv_current + daily_discount_premium
            
            day_date = (row_start_date + pd.Timedelta(days=day)).strftime('%Y-%m-%d')
            
            if day_date in merged_df.columns:
                merged_df.at[idx, day_date] = daily_discount_premium
            
            npv_current = npv_next
    
    # 保存文件
    try:
        output_dir = r"C:\Users\UV665AR\OneDrive - EY\2.KHB\99_Data_Server_out\production\20241231\data\03_report\interim"
        os.makedirs(output_dir, exist_ok=True)
        
        output_path = os.path.join(output_dir, "bondbreakdown.csv")
        merged_df.to_csv(output_path, index=False, encoding='utf-8-sig')
        print(f"Successfully saved merged_df with daily discount/premium to: {output_path}")
    except Exception as e:
        print(f"Warning: Failed to save to interim folder: {e}")
    
    return merged_df

run_yymm = rc.run_yymm
prev_yymm = rc.prev_yymm

test3 = calculate_daily_discount_premium(merged_df, exchange_rate_df, run_yymm, prev_yymm)


Successfully saved merged_df with daily discount/premium to: C:\Users\UV665AR\OneDrive - EY\2.KHB\99_Data_Server_out\production\20241231\data\03_report\interim\bondbreakdown.csv


In [ ]:
import pandas as pd

def calculate_gross_carrying_amount_changes_accrued_interest(
        merged_df, daily_coupon_df, exchange_rate_df):
    """
    Calculate gross carrying amount changes and accrued interest.
    """
    # -------------------------------
    # Step 0: Prepare exchange_rate_map
    # -------------------------------
    exchange_rate_df['DATE_DT'] = pd.to_datetime(exchange_rate_df['DATE'], dayfirst=True)
    exrate_map = dict(zip(exchange_rate_df['DATE_DT'], exchange_rate_df['Exchange Rate']))

    # -------------------------------
    # Step 0.5: Convert merged_df and daily_coupon_df date columns to datetime
    # -------------------------------
    def convert_cols_to_datetime(df):
        new_cols = {}
        for col in df.columns:
            # 仅转换形如 YYYY-MM-DD 或带 '/' 的列
            if isinstance(col, str) and (col.count('-') == 2 or '/' in col):
                try:
                    new_cols[col] = pd.to_datetime(col, dayfirst=True)
                except:
                    new_cols[col] = col
        df.rename(columns=new_cols, inplace=True)
        return df

    merged_df = convert_cols_to_datetime(merged_df)
    daily_coupon_df = convert_cols_to_datetime(daily_coupon_df)

    # -------------------------------
    # Step 1: SUM_DAILY_EXCHANGE
    # -------------------------------
    daily_cols = [col for col in merged_df.columns if isinstance(col, pd.Timestamp)]
    sum_daily_exchange = []

    for idx, row in merged_df.iterrows():
        total = 0
        contract_id = row['CONTRACT_ID']
        cust_id = row['CUST_ID']
        for date_col in daily_cols:
            premium = row[date_col]
            exrate = exrate_map.get(date_col)
            if exrate is None or pd.isna(premium):
                continue
            total += premium * exrate
        sum_daily_exchange.append(total)
    merged_df['SUM_DAILY_EXCHANGE'] = sum_daily_exchange

    # -------------------------------
    # Step 2: SUM_ADJ_EXCHANGE
    # -------------------------------
    sum_adj_exchange = []
    for idx, row in merged_df.iterrows():
        payment_date = row.get('payment_date')
        if pd.notna(payment_date):
            if isinstance(payment_date, str):
                try:
                    payment_date_dt = pd.to_datetime(payment_date, dayfirst=True)
                except:
                    payment_date_dt = pd.to_datetime(payment_date)
            else:
                payment_date_dt = pd.to_datetime(payment_date)
            exrate = exrate_map.get(payment_date_dt, 1.0)
        else:
            exrate = 1.0

        adj_val = (-row.get('DRAWN_BAL_OCY_prev', 0)
                   + row.get('coupon_received', 0)
                   - row.get('accrued_interest', 0)) * exrate
        sum_adj_exchange.append(adj_val)
    merged_df['SUM_ADJ_EXCHANGE'] = sum_adj_exchange

    # -------------------------------
    # Step 3: SUM_DAILY_COUPON
    # -------------------------------
    coupon_cols = [col for col in merged_df.columns if isinstance(col, pd.Timestamp) and col in daily_coupon_df.columns]
    sum_daily_coupon = []

    for idx, row in merged_df.iterrows():
        cust_id = row['CUST_ID']
        contract_id = row['CONTRACT_ID']
        coupon_row = daily_coupon_df[(daily_coupon_df['CUST_ID'] == cust_id) &
                                     (daily_coupon_df['CONTRACT_ID'] == contract_id)]
        total = 0
        if not coupon_row.empty:
            coupon_row = coupon_row.iloc[0]
            for date_col in coupon_cols:
                coupon_val = coupon_row[date_col]
                exrate = exrate_map.get(date_col)
                if exrate is None or pd.isna(coupon_val):
                    continue
                total += coupon_val * exrate
        sum_daily_coupon.append(total)
    merged_df['SUM_DAILY_COUPON'] = sum_daily_coupon

    # -------------------------------
    # Step 4: CHANGES_ACCRUED_INTEREST
    # -------------------------------
    merged_df['CHANGES_ACCRUED_INTEREST'] = (merged_df['SUM_DAILY_EXCHANGE'] +
                                             merged_df['SUM_ADJ_EXCHANGE'] +
                                             merged_df['SUM_DAILY_COUPON'])

    # -------------------------------
    # Step 5: return total sum
    # -------------------------------
    result = merged_df['CHANGES_ACCRUED_INTEREST'].sum()

    # 保存文件
    try:
        output_dir = r"C:\Users\UV665AR\OneDrive - EY\2.KHB\99_Data_Server_out\production\20241231\data\03_report\interim"
        os.makedirs(output_dir, exist_ok=True)
        
        output_path = os.path.join(output_dir, "merged_accrued_interest.csv")
        merged_df.to_csv(output_path, index=False, encoding='utf-8-sig')
        print(f"Successfully saved merged_df with daily discount/premium to: {output_path}")
    except Exception as e:
        print(f"Warning: Failed to save to interim folder: {e}")
    
    return result, merged_df




total_interest, merged_df_updated = calculate_gross_carrying_amount_changes_accrued_interest(
    merged_df, daily_coupon_df, exchange_rate_df
)



Successfully saved merged_df with daily discount/premium to: C:\Users\UV665AR\OneDrive - EY\2.KHB\99_Data_Server_out\production\20241231\data\03_report\interim\merged_accrued_interest.csv


In [ ]:
def calculate_gross_carrying_amount_changes_accrued_interest(merged_df, daily_coupon_df, exchange_rate_df, meas_type):
    # -------------------------------
    # Step 0: Filter by meas_type
    # -------------------------------
    filtered_df = merged_df[merged_df['IFRS9_MEAS_TYPE'] == meas_type].copy()
    if filtered_df.empty:
        return 0.0, merged_df

    # -------------------------------
    # Step 1: Prepare exchange_rate_map
    # -------------------------------
    exchange_rate_df['DATE_DT'] = pd.to_datetime(exchange_rate_df['DATE'], dayfirst=True)
    exrate_map = dict(zip(exchange_rate_df['DATE_DT'], exchange_rate_df['Exchange Rate']))

    # -------------------------------
    # Step 2: Convert date columns to datetime
    # -------------------------------
    def convert_cols_to_datetime(df):
        new_cols = {}
        for col in df.columns:
            if isinstance(col, str) and (col.count('-') == 2 or '/' in col):
                try:
                    new_cols[col] = pd.to_datetime(col, dayfirst=True)
                except:
                    new_cols[col] = col
        df.rename(columns=new_cols, inplace=True)
        return df

    filtered_df = convert_cols_to_datetime(filtered_df)
    daily_coupon_df = convert_cols_to_datetime(daily_coupon_df)

    # -------------------------------
    # Step 3: SUM_DAILY_EXCHANGE
    # -------------------------------
    daily_cols = [col for col in filtered_df.columns if isinstance(col, pd.Timestamp)]
    sum_daily_exchange = []
    for idx, row in filtered_df.iterrows():
        total = 0
        for date_col in daily_cols:
            premium = row[date_col]
            exrate = exrate_map.get(date_col)
            if exrate is None or pd.isna(premium):
                continue
            total += premium * exrate
        sum_daily_exchange.append(total)
    filtered_df['SUM_DAILY_EXCHANGE'] = sum_daily_exchange

    # -------------------------------
    # Step 4: SUM_ADJ_EXCHANGE
    # -------------------------------
    sum_adj_exchange = []
    for idx, row in filtered_df.iterrows():
        payment_date = row.get('payment_date')
        if pd.notna(payment_date):
            if isinstance(payment_date, str):
                try:
                    payment_date_dt = pd.to_datetime(payment_date, dayfirst=True)
                except:
                    payment_date_dt = pd.to_datetime(payment_date)
            else:
                payment_date_dt = pd.to_datetime(payment_date)
            exrate = exrate_map.get(payment_date_dt, 1.0)
        else:
            exrate = 1.0

        adj_val = (-row.get('DRAWN_BAL_OCY_prev', 0)
                   + row.get('coupon_received', 0)
                   - row.get('accrued_interest', 0)) * exrate
        sum_adj_exchange.append(adj_val)
    filtered_df['SUM_ADJ_EXCHANGE'] = sum_adj_exchange

    # -------------------------------
    # Step 5: SUM_DAILY_COUPON
    # -------------------------------
    coupon_cols = [col for col in filtered_df.columns if isinstance(col, pd.Timestamp) and col in daily_coupon_df.columns]
    sum_daily_coupon = []

    for idx, row in filtered_df.iterrows():
        cust_id = row['CUST_ID']
        contract_id = row['CONTRACT_ID']
        coupon_row = daily_coupon_df[(daily_coupon_df['CUST_ID'] == cust_id) &
                                     (daily_coupon_df['CONTRACT_ID'] == contract_id)]
        total = 0
        if not coupon_row.empty:
            coupon_row = coupon_row.iloc[0]
            for date_col in coupon_cols:
                coupon_val = coupon_row[date_col]
                exrate = exrate_map.get(date_col)
                if exrate is None or pd.isna(coupon_val):
                    continue
                total += coupon_val * exrate
        sum_daily_coupon.append(total)
    filtered_df['SUM_DAILY_COUPON'] = sum_daily_coupon

    # -------------------------------
    # Step 6: CHANGES_ACCRUED_INTEREST
    # -------------------------------
    filtered_df['CHANGES_ACCRUED_INTEREST'] = (
        filtered_df['SUM_DAILY_EXCHANGE'] +
        filtered_df['SUM_ADJ_EXCHANGE'] +
        filtered_df['SUM_DAILY_COUPON']
    )

    # -------------------------------
    # Step 7: Save filtered_df
    # -------------------------------
    try:
        output_dir = r"C:\Users\UV665AR\OneDrive - EY\2.KHB\99_Data_Server_out\production\20241231\data\03_report\interim"
        os.makedirs(output_dir, exist_ok=True)
        output_path = os.path.join(output_dir, f"merged_accrued_interest_{meas_type}.csv")
        filtered_df.to_csv(output_path, index=False, encoding='utf-8-sig')
        print(f"Successfully saved merged_df with accrued interest to: {output_path}")
    except Exception as e:
        print(f"Warning: Failed to save to interim folder: {e}")

    # -------------------------------
    # Step 8: Return total sum
    # -------------------------------
    result = filtered_df['CHANGES_ACCRUED_INTEREST'].sum()
    return result


In [98]:
def calculate_gross_carrying_amount_purchase_addition(merged_df, exchange_rate_df, run_yymm, prev_yymm, meas_type):
    # Filter condition: IFRS9_MEAS_TYPE = meas_type and payment_date between RUN_YYMM and PREV_YYMM
    filtered_df = merged_df[
        (merged_df['IFRS9_MEAS_TYPE'] == meas_type) &
        (merged_df['payment_date'] >= str(prev_yymm)) &
        (merged_df['payment_date'] <= str(run_yymm))
    ]
    
    if filtered_df.empty:
        return 0.0
    
    result = 0.0
    
    for _, row in filtered_df.iterrows():
        contract_id = row['CONTRACT_ID']
        cust_id = row['CUST_ID']
        drawn_bal_prev = row['DRAWN_BAL_OCY_prev']
        
        # Get exchange rate for purchased date
        payment_date = row['payment_date']
        payment_exchange_rate = get_exchange_rate(exchange_rate_df, payment_date)
        
        # Calculate: DRAWN_BAL_OCY_prev * EXRATE_purchased / 1000
        amount = drawn_bal_prev * payment_exchange_rate / 1000
        result += amount
    
    return result

test4 = calculate_gross_carrying_amount_purchase_addition(merged_df, exchange_rate_df, run_yymm, prev_yymm, 'AC')
print(test4)

110315360851.60289


In [99]:
def calculate_gross_carrying_amount_matured_repaid(merged_df, exchange_rate_df, run_yymm, prev_yymm, meas_type):
    # Filter condition: IFRS9_MEAS_TYPE = meas_type and MAT_DATE between RUN_YYMM and PREV_YYMM
    # Ensure type consistency for date comparison
    filtered_df = merged_df[
        (merged_df['IFRS9_MEAS_TYPE'] == meas_type) &
        (merged_df['MAT_DATE'] >= str(prev_yymm)) &
        (merged_df['MAT_DATE'] <= str(run_yymm))
    ]
    
    if filtered_df.empty:
        return 0.0
    
    result = 0.0
    
    for _, row in filtered_df.iterrows():
        drawn_bal = row['DRAWN_BAL_OCY']
        mat_date = row['MAT_DATE']
        
        # Get exchange rate for maturity date
        mat_exchange_rate = get_exchange_rate(exchange_rate_df, mat_date)
        
        # Calculate: -sum(DRAWN_BAL_OCY * EXRATE_MAT / 1000)
        amount = -(drawn_bal * mat_exchange_rate / 1000)
        result += amount
    
    return result

test5 = calculate_gross_carrying_amount_matured_repaid(merged_df, exchange_rate_df, run_yymm, prev_yymm, 'AC')
print(test5)

0.0


In [ ]:
def calculate_gross_carrying_amount_decrease_fair_value(merged_df, exchange_rate_df, run_yymm, prev_yymm, meas_type, net_reversal_charge_period):
    # Filter condition: IFRS9_MEAS_TYPE = meas_type
    filtered_df = merged_df[merged_df['IFRS9_MEAS_TYPE'] == meas_type]
    
    if filtered_df.empty:
        return 0.0
    
    result = 0.0
    
    for _, row in filtered_df.iterrows():
        drawn_bal = row['DRAWN_BAL_OCY']
        drawn_bal_prev = row['DRAWN_BAL_OCY_prev']
        
        # Get exchange rates
        run_exchange_rate = get_exchange_rate(exchange_rate_df, run_yymm, '%Y%m%d')
        prev_exchange_rate = get_exchange_rate(exchange_rate_df, prev_yymm, '%Y%m%d')
        
        # Calculate: (DRAWN_BAL_OCY * RUN_YYMM - DRAWN_BAL_OCY_prev * PREV_YYMM) / 1000
        amount = (drawn_bal * run_exchange_rate - drawn_bal_prev * prev_exchange_rate) / 1000
        result += amount
    
    # Add Net (reversal)/charge for the period (Note 16)
    result += net_reversal_charge_period
    
    return result

In [129]:
import pandas as pd
import os

def merge_ecl_and_bond_with_prev(context, param, ecl_result, ecl_result_prev, bond_df, bond_df_prev):
    # -------------------------------
    # Step 0: Define ECL columns to keep
    # -------------------------------
    ecl_cols = ['CONTRACT_ID', 'CUST_ID', 'IFRS9_MEAS_TYPE', 'INIT_DATE', 'MAT_DATE', 
                'DRAWN_BAL_OCY', 'IFRS_AMOUNT_LCY', 'EFF_INT_RT', 'AMORTIZATION_OCY', 
                'IFRS9_PD_12M_MADJ', 'IFRS9_LGD_12M', 'ECL_FINAL_LCY']

    # -------------------------------
    # Step 1: Prepare previous ECL with _prev suffix
    # -------------------------------
    ecl_prev_selected = ecl_result_prev[ecl_cols].copy()
    rename_prev = {col: f"{col}_prev" for col in ecl_prev_selected.columns if col not in ['CONTRACT_ID', 'CUST_ID']}
    ecl_prev_selected = ecl_prev_selected.rename(columns=rename_prev)

    # -------------------------------
    # Step 2: Prepare previous bond with _prev suffix
    # -------------------------------
    bond_prev_renamed = bond_df_prev.copy()
    rename_prev_bond = {col: f"{col}_prev" for col in bond_prev_renamed.columns if col not in ['CONTRACT_ID', 'CUST_ID']}
    bond_prev_renamed = bond_prev_renamed.rename(columns=rename_prev_bond)

    # -------------------------------
    # Step 3: Merge current and previous ECL
    # -------------------------------
    df_curr = ecl_result[ecl_cols].copy()
    merged_ecl = df_curr.merge(ecl_prev_selected, on=['CONTRACT_ID', 'CUST_ID'], how='outer')

    # -------------------------------
    # Step 4: Start with bond_df as base
    # -------------------------------
    merged_df = bond_df.copy()
    merged_df = merged_df.merge(df_curr, on=['CONTRACT_ID', 'CUST_ID'], how='left')
    merged_df = merged_df.merge(ecl_prev_selected, on=['CONTRACT_ID', 'CUST_ID'], how='left')
    merged_df = merged_df.merge(bond_prev_renamed, on=['CONTRACT_ID', 'CUST_ID'], how='left')

    # -------------------------------
    # Step 5: Ensure numeric columns
    # -------------------------------
    numeric_columns = ['DRAWN_BAL_OCY', 'DRAWN_BAL_OCY_prev', 'IFRS_AMOUNT_LCY', 'IFRS_AMOUNT_LCY_prev',
                       'EFF_INT_RT', 'EFF_INT_RT_prev', 'AMORTIZATION_OCY', 'AMORTIZATION_OCY_prev',
                       'interest', 'ECL_FINAL_LCY', 'ECL_FINAL_LCY_prev']
    for col in numeric_columns:
        if col in merged_df.columns:
            merged_df[col] = pd.to_numeric(merged_df[col], errors='coerce')

    # -------------------------------
    # Step 6: Save CSV
    # -------------------------------
    try:
        output_dir = r"C:\Users\UV665AR\OneDrive - EY\2.KHB\99_Data_Server_out\production\20241231\data\03_report\interim"
        os.makedirs(output_dir, exist_ok=True)
        output_path = os.path.join(output_dir, "merged_ecl_bond_full.csv")
        merged_df.to_csv(output_path, index=False, encoding='utf-8-sig')
        print(f"Successfully saved merged full table to: {output_path}")
    except Exception as e:
        print(f"Warning: Failed to save to interim folder: {e}")

    return merged_df


bond_df_prev, daily_coupon_df_prev = load_prev_bond_data(rc, param, prev_yymm)
test6= merge_ecl_and_bond_with_prev(rc, param, ecl_result_filter, ecl_result_filter_prev, bond_df, bond_df_prev)

    

Successfully loaded prev_yymm 20241130 bondbreakdown data from C:\Users\UV665AR\OneDrive - EY\2.KHB\99_Data_Server\production\20241130\data\01_input\bondbreakdown.xlsx


In [130]:
import pandas as pd

def add_ead_columns(merged_df, exchange_rate_df, run_yymm, prev_yymm):
    # -------------------------------
    # Step 0: Prepare exchange rate mapping
    # -------------------------------
    exchange_rate_df['DATE_DT'] = pd.to_datetime(exchange_rate_df['DATE'], dayfirst=True, errors='coerce')
    exrate_map = dict(zip(exchange_rate_df['DATE_DT'], exchange_rate_df['Exchange Rate']))

    run_date = pd.to_datetime(str(run_yymm), format="%Y%m%d")
    prev_date = pd.to_datetime(str(prev_yymm), format="%Y%m%d")

    current_exrate = exrate_map.get(run_date, 1.0)
    prev_exrate = exrate_map.get(prev_date, 1.0)

    # -------------------------------
    # Step 1: Calculate EAD_curr
    # -------------------------------
    for col in ['current_balance', 'accrued_interest_movement', 'unamortized_discount_movement']:
        merged_df[col] = pd.to_numeric(merged_df[col], errors='coerce').fillna(0)

    merged_df['EAD_curr'] = (
        merged_df['current_balance'] + 
        merged_df['accrued_interest_movement'] + 
        merged_df['unamortized_discount_movement']
    ) * current_exrate

    # -------------------------------
    # Step 2: Calculate EAD_prev
    # -------------------------------
    for col in ['current_balance_prev', 'accrued_interest_movement_prev', 'unamortized_discount_movement_prev']:
        if col in merged_df.columns:
            merged_df[col] = pd.to_numeric(merged_df[col], errors='coerce').fillna(0)
        else:
            merged_df[col] = 0

    merged_df['EAD_prev'] = (
        merged_df['current_balance_prev'] +
        merged_df['accrued_interest_movement_prev'] +
        merged_df['unamortized_discount_movement_prev']
    ) * prev_exrate

    # -------------------------------
    # Step 3: Save CSV
    # -------------------------------
    try:
        output_dir = r"C:\Users\UV665AR\OneDrive - EY\2.KHB\99_Data_Server_out\production\20241231\data\03_report\interim"
        os.makedirs(output_dir, exist_ok=True)
        output_path = os.path.join(output_dir, "merged_ecl_bond_full.csv")
        merged_df.to_csv(output_path, index=False, encoding='utf-8-sig')
        print(f"Successfully saved merged full table to: {output_path}")
    except Exception as e:
        print(f"Warning: Failed to save to interim folder: {e}")

    return merged_df

test8 = add_ead_columns(test6, exchange_rate_df, run_yymm, prev_yymm)



Successfully saved merged full table to: C:\Users\UV665AR\OneDrive - EY\2.KHB\99_Data_Server_out\production\20241231\data\03_report\interim\merged_ecl_bond_full.csv


In [ ]:
def calculate_ecl_purchase_new_investments(merged_df, meas_type):
    # -------------------------------
    # Step 0: Filter by IFRS9_MEAS_TYPE
    # -------------------------------
    filtered_df = merged_df[merged_df['IFRS9_MEAS_TYPE'] == meas_type].copy()
    
    if filtered_df.empty:
        return 0.0

    # -------------------------------
    # Step 1: Ensure numeric columns
    # -------------------------------
    for col in ['EAD_curr', 'EAD_prev', 'IFRS9_PD_12M_MADJ', 'IFRS9_LGD_12M']:
        if col in filtered_df.columns:
            filtered_df[col] = pd.to_numeric(filtered_df[col], errors='coerce').fillna(0)
        else:
            filtered_df[col] = 0

    # -------------------------------
    # Step 2: Only rows where EAD_curr > EAD_prev
    # -------------------------------
    filtered_df = filtered_df[filtered_df['EAD_curr'] > filtered_df['EAD_prev']]

    if filtered_df.empty:
        return 0.0

    # -------------------------------
    # Step 3: Calculate ECL contribution
    # -------------------------------
    ecl_values = (filtered_df['EAD_curr'] - filtered_df['EAD_prev']) * \
                 filtered_df['IFRS9_PD_12M_MADJ'] * \
                 filtered_df['IFRS9_LGD_12M'] / 1000

    total_ecl = ecl_values.sum()

    return total_ecl

In [136]:
def calculate_ecl_changes_input_ecl_calculation(merged_df, meas_type='FVOCI'):
    # -------------------------------
    # Step 0: Filter by IFRS9_MEAS_TYPE
    # -------------------------------
    filtered_df = merged_df[merged_df['IFRS9_MEAS_TYPE'] == meas_type].copy()
    
    if filtered_df.empty:
        return 0.0

    # -------------------------------
    # Step 1: Ensure numeric columns
    # -------------------------------
    for col in ['EAD_prev', 'IFRS9_PD_12M_MADJ', 'IFRS9_LGD_12M', 'IFRS9_PD_12M_MADJ_prev', 'IFRS9_LGD_12M_prev']:
        if col in filtered_df.columns:
            filtered_df[col] = pd.to_numeric(filtered_df[col], errors='coerce').fillna(0)
        else:
            filtered_df[col] = 0

    # -------------------------------
    # Step 2: Calculate ECL contribution
    # -------------------------------
    ecl_values = filtered_df['EAD_prev'] * (
        filtered_df['IFRS9_PD_12M_MADJ'] * filtered_df['IFRS9_LGD_12M'] -
        filtered_df['IFRS9_PD_12M_MADJ_prev'] * filtered_df['IFRS9_LGD_12M_prev']
    )

    total_ecl = ecl_values.sum()

    return total_ecl

test10 = calculate_ecl_changes_input_ecl_calculation(test8, 'AC')
print(test10)

0.0


In [138]:
def set_interim_path(context):
    global interim_path
    interim_dir = Path(context.resultPath) / "interim"
    interim_dir.mkdir(parents=True, exist_ok=True)
    interim_path = interim_dir
    return interim_path

test11 = set_interim_path(rc)
print(test11)


C:\Users\UV665AR\OneDrive - EY\2.KHB\99_Data_Server_out\production\20241231\data\02_result\interim
